# 加载库函数

In [1]:
import numpy as np
import scipy.sparse as sp
import networkx as nx
import networkx.algorithms.isomorphism as iso
import itertools
import os.path
import subprocess
import torch.nn.functional as F
import pickle
import os
import scipy.sparse as sp
import torch
import json
import scipy.io as sio
from torch.nn import Embedding
#import scipy.sparse as spsparse
from torch.nn import Parameter
import torch
import torch.nn as nn
from scipy.sparse import csr_matrix
import torch.nn.functional as F
import argparse
from sklearn.metrics import average_precision_score, roc_auc_score, accuracy_score, f1_score, precision_score, recall_score

# 加载数据，数据处理

In [2]:
"""
Disclaimer: functions defined from lines 15 to 36 in this file come from 
tkipf/gae original repository on Graph Autoencoders. Moreover, the
mask_test_edges_general_link_prediction function is borrowed from 
philipjackson's mask_test_edges pull request on this same repository.
"""

def sparse_to_tuple(sparse_mx):
    if not sp.isspmatrix_coo(sparse_mx):
        sparse_mx = sparse_mx.tocoo()
    coords = np.vstack((sparse_mx.row, sparse_mx.col)).transpose()
    values = sparse_mx.data
    shape = sparse_mx.shape
    return coords, values, shape

In [3]:
# Task 3 - Bidirectional Link Prediction
def mask_test_edges_bidirectional_link_prediction(adj):
    """
    Task 3: Bidirectionality Prediction: get Train and Test sets

    :param adj: complete sparse adjacency matrix of the graph
    :return: train incomplete adjacency matrix and test sets
    """

    # Remove diagonal elements
    adj = adj - sp.dia_matrix((adj.diagonal()[None, :], [0]), shape = adj.shape)
    adj.eliminate_zeros()
    val_edges, val_edges_false, test_edges, test_edges_false = None, None, None, None

    ## Unidirectional edges
    adj_sym = (adj + adj.T).sign()
    adj_tilde = (adj_sym - adj).T
    adj_tilde.eliminate_zeros()

    ## Bidirectional edges (they usually are few, so they are all in test set)
    adj_sym_ones = adj - adj_tilde
    adj_sym_ones.eliminate_zeros()

    # Positive node pairs in test set (bidirectional edges)
    test_edges, _, _ = sparse_to_tuple(adj_sym_ones)
    test_edges = test_edges[test_edges[:,1] > test_edges[:,0],:]

    # Negative node pairs in test set (unidirectional edges)
    test_edges_false, _, _ = sparse_to_tuple(adj_tilde)
    test_edges_false = test_edges_false[test_edges_false[:,0] > test_edges_false[:,1],:]
    test_edges_false = np.fliplr(test_edges_false)
    # Sampling of negative node pairs
    edges_negative_idx = np.arange(test_edges_false.shape[0])
    np.random.shuffle(edges_negative_idx)
    test_edges_false_idx = edges_negative_idx[:test_edges.shape[0]]
    test_edges_false = test_edges_false[test_edges_false_idx]

    # Get training incomplete adjacency matrix
    # 1 of the 2 directions of each bidirectional edge is masked
    adj_train = (adj - sp.triu(adj_sym_ones))
    all_train_edges = adj_train.nonzero()
    out_train_edges = [[all_train_edges[0][i],all_train_edges[1][i]] for i in range(all_train_edges[0].shape[0])]
    # Validation set: not implemented for Task 2
    # val_edges and val_edges_false are None
    return np.array(out_train_edges), val_edges_false, test_edges, test_edges_false

In [4]:
def load_data(dataset):
    """ Load datasets from text files

    :param dataset: 'cora', 'citeseer' or 'google' graph dataset.
    :return: n*n sparse adjacency matrix and n*f node-level feature matrix

    Note: in this paper, all three datasets are assumed to be featureless.
    As a consequence, the feature matrix is the identity matrix I_n.
    """
    if dataset == 'cora':
        adj = nx.adjacency_matrix(nx.read_edgelist("directed_data/cora.cites",
                                                   delimiter='\t',
                                                   create_using=nx.DiGraph()))
        # Transpose the adjacency matrix, as Cora raw dataset comes with a
        # <ID of cited paper> <ID of citing paper> edgelist format.
        adj = adj.T
        features = sp.identity(adj.shape[0])

    elif dataset == 'citeseer':
        adj = nx.adjacency_matrix(nx.read_edgelist("directed_data/citeseer.cites",
                                                   delimiter='\t',
                                                   create_using=nx.DiGraph()))
        # Transpose the adjacency matrix, as Citeseer raw dataset comes with a
        # <ID of cited paper> <ID of citing paper> edgelist format.
        adj = adj.T
        features = sp.identity(adj.shape[0])

    elif dataset in ['Wisconsin', 'Texas', 'Cornell']:
        from torch_geometric.datasets import WebKB
        from scipy import sparse

        graph_data = WebKB(root='directed_data/', name=dataset)
        data = graph_data[0]

        edges = data.edge_index
        nodes_number = data.x.shape[0]
        graph_np = np.zeros((nodes_number, nodes_number))
        for i in range(edges.shape[1]):
            graph_np[edges[0, i], edges[1, i]] = 1
    
        adj = sparse.csr_matrix(graph_np)
        
        features = sp.identity(adj.shape[0])

    else:
        raise ValueError('Undefined dataset!')

    return adj, features

In [5]:
def get_graph(DATASET='cora'):
    adj_init, features = load_data(DATASET)

    #train_edges, val_edges, val_edges_false, test_edges, test_edges_false = mask_test_edges_biased_negative_samples(adj_init)
    train_edges, val_edges_false, test_edges, test_edges_false = mask_test_edges_bidirectional_link_prediction(adj_init)
    
    all_negative_edges = np.argwhere(adj_init.toarray()==0)
    
    all_train_edges_false = np.delete(all_negative_edges, test_edges_false, axis = 0)
    
    if DATASET == 'cora':
        nodes_number = 2708
    elif DATASET == 'citeseer':
        nodes_number = 3327
    elif DATASET == 'Wisconsin':
        nodes_number = 251
    elif DATASET == 'Texas':
        nodes_number = 183
    elif DATASET == 'Cornell':
        nodes_number = 183
        
    ################## 处理测试集  #############################
    test_posi_mask = [test_edges[i][0]*nodes_number+test_edges[i][1] for i in range(len(test_edges))]
    test_false_mask = [test_edges_false[i][0]*nodes_number+test_edges_false[i][1] for i in range(len(test_edges_false))]
    test_mask = test_posi_mask + test_false_mask
    test_mask = torch.tensor(test_mask)
    test_edge_true_or_false = torch.zeros(len(test_mask))
    test_edge_true_or_false[:len(test_posi_mask)] = 1
    
    ############### 处理训练集  ################################
    negative_index = np.random.choice(range(all_train_edges_false.shape[0]), train_edges.shape[0]*NEGATIVE_SAMPLE, replace=False)
    train_edges_false = all_train_edges_false[negative_index]
    train_posi_mask = [train_edges[i][0]*nodes_number+train_edges[i][1] for i in range(len(train_edges))]
    train_false_mask = [train_edges_false[i][0]*nodes_number+train_edges_false[i][1] for i in range(len(train_edges_false))]
    train_mask = train_posi_mask + train_false_mask
    train_mask = torch.tensor(train_mask)
    #train_mask = torch.tensor(train_posi_mask)
    train_edge_true_or_false = torch.zeros(len(train_mask))
    train_edge_true_or_false[:len(train_posi_mask)] = 1
    
    return train_edge_true_or_false, test_edge_true_or_false, train_mask.long(), test_mask.long(), nodes_number

# 定义模型

In [7]:
def predict_acc(recons_edges, true_edges):
    predict_graph = recons_edges
    predict_edges = np.array(predict_graph)
    ap = average_precision_score(true_edges, predict_edges)
    auc = roc_auc_score(true_edges, predict_edges)
    print("AP SCORE： ", ap)
    print("AUC SCORE: ", auc)
    return ap, auc

################  更新节点内容  #######################################
class reconstruction_graph(nn.Module):
    """给定类簇质心，更新节点嵌入"""
    def __init__(self, context_embedding):
        super(reconstruction_graph, self).__init__()
        self.context_embedding = Parameter(context_embedding)
        
    def forward(self, importance_embedding):
        power = float(ALPHA + 1) / 2    # 计算幂
        
        # 计算节点之间的距离 ： 值越小越有利于边的形成
        nodes_distance = torch.norm(self.context_embedding[:, None]-self.context_embedding, dim=2, p=2)
        nodes_distance = 1.0 + nodes_distance
        
        # 计算节点之间的重要性：值越大越有利于边的形成
        norm_squared = torch.sum((self.context_embedding.unsqueeze(1) - importance_embedding)**2, 2)
        nodes_importance = N / (1.0 + norm_squared) + 1.0
        importance_prod = torch.mm(nodes_importance, nodes_importance.t())   # 分子
        importance_norm = torch.norm(nodes_importance, p=2, dim=1).unsqueeze(0)   #分母(每行的长度)
        importance_norm = importance_norm**2
        importance_similar = importance_prod.div(importance_norm.t())
        
        # 计算边的形成概率
        nodes_similar = torch.div(beta*importance_similar, nodes_distance)
        return 2*sigmoid_modle(nodes_similar)-1
        #return torch.exp(-nodes_similar)
        
    
class update_nodes_embedding(nn.Module):
    def __init__(self, context_embedding):
        super(update_nodes_embedding, self).__init__()
        self.reconstruction_module = reconstruction_graph(context_embedding)     # 更新节点嵌入
        self.optimizer = torch.optim.SGD(params=self.reconstruction_module.parameters(), lr=LR, momentum=MOMENTUM)
        self.loss_function = torch.nn.MSELoss(reduction='sum')
        
    def forward(self, train_edges, test_edges, edge_train_mask, edge_test_mask, importance_embedding):
        self.reconstruction_module.train()
        for epoch in range(EPOCH):
            self.optimizer.zero_grad()
            graph_reconstruction = self.reconstruction_module(importance_embedding)
            graph_train = torch.take(graph_reconstruction, edge_train_mask)
            loss = self.loss_function(train_edges, graph_train)
            loss.backward()
            self.optimizer.step()
            print(f'Epoch: {epoch:02d}, Loss: {loss.item():.4f}')
        recons_test_edges = torch.take(graph_reconstruction, edge_test_mask).detach()
        ap, auc = predict_acc(recons_test_edges, test_edges)
        return self.reconstruction_module.context_embedding.detach(), ap, auc

####################  更新节点重要性  ######################################
class reconstruction_graph_importance(nn.Module):
    """给定类簇质心，更新节点嵌入"""
    def __init__(self, importance_embedding):
        super(reconstruction_graph_importance, self).__init__()
        self.importance_embedding = Parameter(importance_embedding)
        
    def forward(self, context_embedding):
        power = float(ALPHA + 1) / 2    # 计算幂
        
        # 计算节点之间的距离 ： 值越小越有利于边的形成
        nodes_distance = torch.norm(context_embedding[:, None]-context_embedding, dim=2, p=2)
        nodes_distance = 1.0 + nodes_distance
        
        # 计算节点之间的重要性：值越大越有利于边的形成
        norm_squared = torch.sum((context_embedding.unsqueeze(1) - self.importance_embedding)**2, 2)
        nodes_importance = N / (1.0 + norm_squared) + 1.0
        importance_prod = torch.mm(nodes_importance, nodes_importance.t())   # 分子
        importance_norm = torch.norm(nodes_importance, p=2, dim=1).unsqueeze(0)   #分母(每行的长度)
        importance_norm = importance_norm**2
        importance_similar = importance_prod.div(importance_norm.t())
        
        # 计算边的形成概率
        nodes_similar = torch.div(beta*importance_similar, nodes_distance)
        return 2*sigmoid_modle(nodes_similar)-1
        #return torch.exp(-nodes_similar)

class update_nodes_embedding_importance(nn.Module):
    def __init__(self, importance_embedding):
        super(update_nodes_embedding_importance, self).__init__()
        self.reconstruction_module = reconstruction_graph_importance(importance_embedding)     # 更新节点嵌入
        self.optimizer = torch.optim.SGD(params=self.reconstruction_module.parameters(), lr=LR, momentum=MOMENTUM)
        self.loss_function = torch.nn.MSELoss(reduction='sum')

    def forward(self, train_edges, test_edges, edge_train_mask, edge_test_mask, context_embedding):
        self.reconstruction_module.train()
        for epoch in range(EPOCH):
            self.optimizer.zero_grad()
            graph_reconstruction = self.reconstruction_module(context_embedding)
            graph_train = torch.take(graph_reconstruction, edge_train_mask)
            loss = self.loss_function(train_edges, graph_train)
            loss.backward()
            self.optimizer.step()
            print(f'Epoch: {epoch:02d}, Loss: {loss.item():.4f}')
        recons_test_edges = torch.take(graph_reconstruction, edge_test_mask).detach()
        ap, auc = predict_acc(recons_test_edges, test_edges)
        return self.reconstruction_module.importance_embedding.detach(), ap, auc

In [11]:
context_embedding_dim = 12
importance_embedding_dim = 48
beta = 4.2   # 原来是4.2
ALPHA = 1.0
LR = 0.05
MOMENTUM = 0.9
EPOCH = 5
N = 29.0    # 目前25最佳
NEGATIVE_SAMPLE = 6
sigmoid_modle = torch.nn.Sigmoid()

dataset = 'citeseer'

all_ap_first_best_auc = []
all_ap_first_best_ap = []

all_auc_first_best_auc = []
all_auc_first_best_ap = []

for i in range(10):
    train_edge, test_edge, train_mask, test_mask, nodes_number = get_graph(DATASET=dataset)
    
    context_embedding = Embedding(nodes_number, context_embedding_dim, sparse=True)      # 时刻为0时给定随机的嵌入
    nodes_context_embeddings = context_embedding.weight.detach()

    importance_embedding = Embedding(importance_embedding_dim, context_embedding_dim, sparse=True)      # 时刻为0时给定随机的嵌入
    nodes_importance_embeddings = importance_embedding.weight.detach()

    auc_first_best_auc = 0
    auc_first_best_ap = 0
    ap_first_best_auc = 0
    ap_first_best_ap = 0

    for module_epoch in range(100):
        print("######################### 模块循环 ： %d ##########################"%module_epoch)
        update_nodes_module = update_nodes_embedding(nodes_context_embeddings)
        nodes_context_embeddings, ap, auc = update_nodes_module(train_edge, test_edge, train_mask, test_mask, nodes_importance_embeddings)
        if auc > auc_first_best_auc:
            auc_first_best_auc = auc
            auc_first_best_ap = ap
        if ap > ap_first_best_ap:
            ap_first_best_auc = auc
            ap_first_best_ap = ap
        update_nodes_importance_module = update_nodes_embedding_importance(nodes_importance_embeddings)
        nodes_importance_embeddings, ap, auc = update_nodes_importance_module(train_edge, test_edge, train_mask, test_mask, nodes_context_embeddings)
        if auc > auc_first_best_auc:
            auc_first_best_auc = auc
            auc_first_best_ap = ap
        if ap > ap_first_best_ap:
            ap_first_best_auc = auc
            ap_first_best_ap = ap
    
    all_ap_first_best_ap.append(ap_first_best_ap)
    all_ap_first_best_auc.append(ap_first_best_auc)
    
    all_auc_first_best_ap.append(auc_first_best_ap)
    all_auc_first_best_auc.append(auc_first_best_auc)
    
print("AP FIRST MEAN AP : ", np.array(all_ap_first_best_ap).mean())
print("AP FIRST STD AP : ", np.array(all_ap_first_best_ap).std())

print("AP FIRST MEAN AUC : ", np.array(all_ap_first_best_auc).mean())
print("AP FIRST STD AUC : ", np.array(all_ap_first_best_auc).std())

print("AUC FIRST MEAN AP : ", np.array(auc_first_best_ap).mean())
print("AUC FIRST STD AP : ", np.array(auc_first_best_ap).std())

print("AUC FIRST MEAN AUC : ", np.array(auc_first_best_auc).mean())
print("AUC FIRST STD AUC : ", np.array(auc_first_best_auc).std())

######################### 模块循环 ： 0 ##########################
Epoch: 00, Loss: 5436.5986
Epoch: 01, Loss: 5378.4990
Epoch: 02, Loss: 5272.7490
Epoch: 03, Loss: 5140.6328
Epoch: 04, Loss: 5000.8408
AP SCORE：  0.6060640723059533
AUC SCORE:  0.5583545918367346
Epoch: 00, Loss: 4855.7397
Epoch: 01, Loss: 4853.7402
Epoch: 02, Loss: 4850.4927
Epoch: 03, Loss: 4847.0430
Epoch: 04, Loss: 4844.1089
AP SCORE：  0.6143189219708468
AUC SCORE:  0.5596301020408163
######################### 模块循环 ： 1 ##########################
Epoch: 00, Loss: 4841.5669
Epoch: 01, Loss: 4805.8613
Epoch: 02, Loss: 4743.9067
Epoch: 03, Loss: 4668.0112
Epoch: 04, Loss: 4583.8135
AP SCORE：  0.611296450156906
AUC SCORE:  0.5653698979591837
Epoch: 00, Loss: 4489.9478
Epoch: 01, Loss: 4485.3252
Epoch: 02, Loss: 4480.5503
Epoch: 03, Loss: 4476.7861
Epoch: 04, Loss: 4473.0376
AP SCORE：  0.6058305219417597
AUC SCORE:  0.564094387755102
######################### 模块循环 ： 2 ##########################
Epoch: 00, Loss: 4470.0674
Epoch

Epoch: 00, Loss: 3118.9626
Epoch: 01, Loss: 3116.4326
Epoch: 02, Loss: 3111.6504
Epoch: 03, Loss: 3104.8857
Epoch: 04, Loss: 3096.3970
AP SCORE：  0.6377124580843541
AUC SCORE:  0.6380739795918368
Epoch: 00, Loss: 3086.4275
Epoch: 01, Loss: 3086.3599
Epoch: 02, Loss: 3086.2500
Epoch: 03, Loss: 3086.1313
Epoch: 04, Loss: 3086.0288
AP SCORE：  0.6280911179096159
AUC SCORE:  0.6377551020408163
######################### 模块循环 ： 19 ##########################
Epoch: 00, Loss: 3085.9573
Epoch: 01, Loss: 3083.5154
Epoch: 02, Loss: 3078.9016
Epoch: 03, Loss: 3072.3804
Epoch: 04, Loss: 3064.2087
AP SCORE：  0.6297027281444111
AUC SCORE:  0.637436224489796
Epoch: 00, Loss: 3054.6313
Epoch: 01, Loss: 3054.5762
Epoch: 02, Loss: 3054.4897
Epoch: 03, Loss: 3054.4014
Epoch: 04, Loss: 3054.3318
AP SCORE：  0.6297957267609288
AUC SCORE:  0.6377551020408163
######################### 模块循环 ： 20 ##########################
Epoch: 00, Loss: 3054.2886
Epoch: 01, Loss: 3051.9609
Epoch: 02, Loss: 3047.5679
Epoch: 03,

Epoch: 01, Loss: 2683.8389
Epoch: 02, Loss: 2681.2356
Epoch: 03, Loss: 2677.5454
Epoch: 04, Loss: 2672.8989
AP SCORE：  0.6670182513493049
AUC SCORE:  0.6712372448979591
Epoch: 00, Loss: 2667.4175
Epoch: 01, Loss: 2667.3928
Epoch: 02, Loss: 2667.3604
Epoch: 03, Loss: 2667.3367
Epoch: 04, Loss: 2667.3225
AP SCORE：  0.6650559149574861
AUC SCORE:  0.6709183673469388
######################### 模块循环 ： 37 ##########################
Epoch: 00, Loss: 2667.3125
Epoch: 01, Loss: 2665.9624
Epoch: 02, Loss: 2663.4067
Epoch: 03, Loss: 2659.7827
Epoch: 04, Loss: 2655.2178
AP SCORE：  0.66238737353638
AUC SCORE:  0.6699617346938775
Epoch: 00, Loss: 2649.8296
Epoch: 01, Loss: 2649.8057
Epoch: 02, Loss: 2649.7739
Epoch: 03, Loss: 2649.7498
Epoch: 04, Loss: 2649.7349
AP SCORE：  0.6608470497017006
AUC SCORE:  0.6693239795918368
######################### 模块循环 ： 38 ##########################
Epoch: 00, Loss: 2649.7241
Epoch: 01, Loss: 2648.3960
Epoch: 02, Loss: 2645.8818
Epoch: 03, Loss: 2642.3167
Epoch: 04, 

Epoch: 02, Loss: 2402.9150
Epoch: 03, Loss: 2400.2271
Epoch: 04, Loss: 2396.8413
AP SCORE：  0.6599759514889134
AUC SCORE:  0.6830357142857142
Epoch: 00, Loss: 2392.8450
Epoch: 01, Loss: 2392.8242
Epoch: 02, Loss: 2392.7986
Epoch: 03, Loss: 2392.7778
Epoch: 04, Loss: 2392.7625
AP SCORE：  0.6599307143341735
AUC SCORE:  0.6827168367346939
######################### 模块循环 ： 55 ##########################
Epoch: 00, Loss: 2392.7493
Epoch: 01, Loss: 2391.7642
Epoch: 02, Loss: 2389.8992
Epoch: 03, Loss: 2387.2559
Epoch: 04, Loss: 2383.9253
AP SCORE：  0.6613484311706749
AUC SCORE:  0.6833545918367346
Epoch: 00, Loss: 2379.9929
Epoch: 01, Loss: 2379.9709
Epoch: 02, Loss: 2379.9443
Epoch: 03, Loss: 2379.9229
Epoch: 04, Loss: 2379.9060
AP SCORE：  0.6630545033371051
AUC SCORE:  0.6839923469387754
######################### 模块循环 ： 56 ##########################
Epoch: 00, Loss: 2379.8928
Epoch: 01, Loss: 2378.9211
Epoch: 02, Loss: 2377.0813
Epoch: 03, Loss: 2374.4734
Epoch: 04, Loss: 2371.1875
AP SCORE：

Epoch: 03, Loss: 2192.6409
Epoch: 04, Loss: 2189.9470
AP SCORE：  0.6610225910552245
AUC SCORE:  0.6871811224489796
Epoch: 00, Loss: 2186.7627
Epoch: 01, Loss: 2186.7466
Epoch: 02, Loss: 2186.7305
Epoch: 03, Loss: 2186.7217
Epoch: 04, Loss: 2186.7161
AP SCORE：  0.6607637918833819
AUC SCORE:  0.6871811224489796
######################### 模块循环 ： 73 ##########################
Epoch: 00, Loss: 2186.7104
Epoch: 01, Loss: 2185.9231
Epoch: 02, Loss: 2184.4336
Epoch: 03, Loss: 2182.3193
Epoch: 04, Loss: 2179.6533
AP SCORE：  0.6612555167102945
AUC SCORE:  0.6865433673469387
Epoch: 00, Loss: 2176.5015
Epoch: 01, Loss: 2176.4856
Epoch: 02, Loss: 2176.4697
Epoch: 03, Loss: 2176.4609
Epoch: 04, Loss: 2176.4558
AP SCORE：  0.6605116288152438
AUC SCORE:  0.685905612244898
######################### 模块循环 ： 74 ##########################
Epoch: 00, Loss: 2176.4504
Epoch: 01, Loss: 2175.6709
Epoch: 02, Loss: 2174.1958
Epoch: 03, Loss: 2172.1028
Epoch: 04, Loss: 2169.4629
AP SCORE：  0.6613873235676929
AUC SCO

...

Epoch: 01, Loss: 2955.6006
Epoch: 02, Loss: 2955.5176
Epoch: 03, Loss: 2955.4236
Epoch: 04, Loss: 2955.3364
AP SCORE：  0.6338125250836486
AUC SCORE:  0.6450892857142857
######################### 模块循环 ： 24 ##########################
Epoch: 00, Loss: 2955.2646
Epoch: 01, Loss: 2953.3242
Epoch: 02, Loss: 2949.6562
Epoch: 03, Loss: 2944.4678
Epoch: 04, Loss: 2937.9565
AP SCORE：  0.6348816326896907
AUC SCORE:  0.6454081632653061
Epoch: 00, Loss: 2930.3064
Epoch: 01, Loss: 2930.2629
Epoch: 02, Loss: 2930.1948
Epoch: 03, Loss: 2930.1235
Epoch: 04, Loss: 2930.0632
AP SCORE：  0.6363407166035935
AUC SCORE:  0.647640306122449
######################### 模块循环 ： 25 ##########################
Epoch: 00, Loss: 2930.0193
Epoch: 01, Loss: 2928.1545
Epoch: 02, Loss: 2924.6299
Epoch: 03, Loss: 2919.6411
Epoch: 04, Loss: 2913.3770
AP SCORE：  0.6377928352339944
AUC SCORE:  0.6498724489795918
Epoch: 00, Loss: 2906.0122
Epoch: 01, Loss: 2905.9768
Epoch: 02, Loss: 2905.9238
Epoch: 03, Loss: 2905.8679
Epoch: 04,

Epoch: 02, Loss: 2594.2556
Epoch: 03, Loss: 2594.2246
Epoch: 04, Loss: 2594.2036
AP SCORE：  0.6429407241379659
AUC SCORE:  0.6444515306122448
######################### 模块循环 ： 42 ##########################
Epoch: 00, Loss: 2594.1904
Epoch: 01, Loss: 2592.9307
Epoch: 02, Loss: 2590.5461
Epoch: 03, Loss: 2587.1636
Epoch: 04, Loss: 2582.9021
AP SCORE：  0.6442235746620912
AUC SCORE:  0.6438137755102041
Epoch: 00, Loss: 2577.8701
Epoch: 01, Loss: 2577.8455
Epoch: 02, Loss: 2577.8098
Epoch: 03, Loss: 2577.7773
Epoch: 04, Loss: 2577.7537
AP SCORE：  0.6449997655983015
AUC SCORE:  0.6444515306122449
######################### 模块循环 ： 43 ##########################
Epoch: 00, Loss: 2577.7375
Epoch: 01, Loss: 2576.4944
Epoch: 02, Loss: 2574.1414
Epoch: 03, Loss: 2570.8047
Epoch: 04, Loss: 2566.6018
AP SCORE：  0.6463726183120526
AUC SCORE:  0.6441326530612245
Epoch: 00, Loss: 2561.6394
Epoch: 01, Loss: 2561.6128
Epoch: 02, Loss: 2561.5732
Epoch: 03, Loss: 2561.5352
Epoch: 04, Loss: 2561.5046
AP SCORE：

Epoch: 03, Loss: 2330.5461
Epoch: 04, Loss: 2330.5364
AP SCORE：  0.6514383863442862
AUC SCORE:  0.6355229591836735
######################### 模块循环 ： 60 ##########################
Epoch: 00, Loss: 2330.5317
Epoch: 01, Loss: 2329.5449
Epoch: 02, Loss: 2327.6765
Epoch: 03, Loss: 2325.0264
Epoch: 04, Loss: 2321.6875
AP SCORE：  0.649861174910274
AUC SCORE:  0.6348852040816326
Epoch: 00, Loss: 2317.7451
Epoch: 01, Loss: 2317.7266
Epoch: 02, Loss: 2317.7019
Epoch: 03, Loss: 2317.6836
Epoch: 04, Loss: 2317.6729
AP SCORE：  0.6508818021951867
AUC SCORE:  0.6358418367346939
######################### 模块循环 ： 61 ##########################
Epoch: 00, Loss: 2317.6680
Epoch: 01, Loss: 2316.6931
Epoch: 02, Loss: 2314.8467
Epoch: 03, Loss: 2312.2280
Epoch: 04, Loss: 2308.9277
AP SCORE：  0.6530375325307557
AUC SCORE:  0.6383928571428571
Epoch: 00, Loss: 2305.0295
Epoch: 01, Loss: 2305.0112
Epoch: 02, Loss: 2304.9878
Epoch: 03, Loss: 2304.9695
Epoch: 04, Loss: 2304.9602
AP SCORE：  0.6540121303098648
AUC SCO

Epoch: 04, Loss: 2123.0742
AP SCORE：  0.6447029991696476
AUC SCORE:  0.6374362244897959
######################### 模块循环 ： 78 ##########################
Epoch: 00, Loss: 2123.0723
Epoch: 01, Loss: 2122.3047
Epoch: 02, Loss: 2120.8525
Epoch: 03, Loss: 2118.7942
Epoch: 04, Loss: 2116.2026
AP SCORE：  0.6468339686941459
AUC SCORE:  0.639030612244898
Epoch: 00, Loss: 2113.1443
Epoch: 01, Loss: 2113.1299
Epoch: 02, Loss: 2113.1140
Epoch: 03, Loss: 2113.1057
Epoch: 04, Loss: 2113.1028
AP SCORE：  0.6465609905462967
AUC SCORE:  0.6387117346938775
######################### 模块循环 ： 79 ##########################
Epoch: 00, Loss: 2113.1003
Epoch: 01, Loss: 2112.3455
Epoch: 02, Loss: 2110.9180
Epoch: 03, Loss: 2108.8945
Epoch: 04, Loss: 2106.3462
AP SCORE：  0.6467503857924415
AUC SCORE:  0.6387117346938774
Epoch: 00, Loss: 2103.3391
Epoch: 01, Loss: 2103.3250
Epoch: 02, Loss: 2103.3093
Epoch: 03, Loss: 2103.3008
Epoch: 04, Loss: 2103.2976
AP SCORE：  0.6464672847471453
AUC SCORE:  0.6383928571428571
###

Epoch: 00, Loss: 1964.0197
Epoch: 01, Loss: 1963.4193
Epoch: 02, Loss: 1962.2837
Epoch: 03, Loss: 1960.6720
Epoch: 04, Loss: 1958.6393
AP SCORE：  0.659084139266311
AUC SCORE:  0.6457270408163265
Epoch: 00, Loss: 1956.2346
Epoch: 01, Loss: 1956.2240
Epoch: 02, Loss: 1956.2130
Epoch: 03, Loss: 1956.2085
Epoch: 04, Loss: 1956.2063
AP SCORE：  0.655754541373763
AUC SCORE:  0.6460459183673469
######################### 模块循环 ： 97 ##########################
Epoch: 00, Loss: 1956.2037
Epoch: 01, Loss: 1955.6101
Epoch: 02, Loss: 1954.4863
Epoch: 03, Loss: 1952.8912
Epoch: 04, Loss: 1950.8792
AP SCORE：  0.6590184375482491
AUC SCORE:  0.6470025510204082
Epoch: 00, Loss: 1948.4985
Epoch: 01, Loss: 1948.4878
Epoch: 02, Loss: 1948.4777
Epoch: 03, Loss: 1948.4735
Epoch: 04, Loss: 1948.4716
AP SCORE：  0.6557906027396269
AUC SCORE:  0.6466836734693877
######################### 模块循环 ： 98 ##########################
Epoch: 00, Loss: 1948.4689
Epoch: 01, Loss: 1947.8811
Epoch: 02, Loss: 1946.7683
Epoch: 03, 